<a href="https://colab.research.google.com/github/wolfzxcv/ml-examples/blob/master/TSP_using_GA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Key Components of a Genetic Algorithm**

* Selection: Choosing individuals from the population to create offspring.
* Crossover: Combining parts of two individuals to create a new individual.
* Mutation: Randomly changes genes to introduce diversity.

**Bonus**
* Elitism: Retains top individuals in the next generation to preserve the best solutions.

In [73]:
from random import randint, choice

# Number of cities in TSP
V = 5

# Names of the 5 cities
GENES = "ABCDE"

# Initial population size (potential solutions) for the algorithm
POP_SIZE = 10  # Example with an odd number

# Max number of generations
max_gen = 20

In [74]:
# Individual solution (a possible path)
class Individual:
    def __init__(self, gnome, fitness):
        self.gnome = gnome  # A string representing the path (e.g., "012340")
        self.fitness = fitness  # The total distance of the path

# Generate a random number between start and end-1
def rand_num(start, end):
    return randint(start, end - 1)  # Adjust for Python's zero-based indexing

# Generates a valid initial path (gnome) starting and ending at the first city
def create_gnome():
    gnome = str(choice(range(V)))
    while len(gnome) < V:
        temp = rand_num(0, V)
        if str(temp) not in gnome:
            gnome += str(temp)
    return gnome

# Perform crossover between two parents to create two offspring
def crossover(parent1, parent2):
    geneA = rand_num(1, V)
    geneB = rand_num(1, V)

    startGene = min(geneA, geneB)
    endGene = max(geneA, geneB)

    child1_p1 = parent1.gnome[startGene:endGene]
    child2_p1 = parent2.gnome[startGene:endGene]

    child1_p2 = [gene for gene in parent2.gnome if gene not in child1_p1]
    child2_p2 = [gene for gene in parent1.gnome if gene not in child2_p1]

    child1 = child1_p1 + ''.join(child1_p2)
    child2 = child2_p1 + ''.join(child2_p2)

    return child1, child2

# Mutates a gnome by swapping two random cities in the path
def mutated_gene(gnome):
    gnome = list(gnome)
    while True:
        r = rand_num(1, V)
        r1 = rand_num(1, V)
        if r1 != r:
            gnome[r], gnome[r1] = gnome[r1], gnome[r]
            break
    return ''.join(gnome)

# Calculates the fitness of a gnome by summing the distances between continuous cities in the path
def calculate_fitness(gnome):
    # A very high value to represent no direct path between cities
    INT_MAX = 2147483647

    # distance_matrix represents the distances between cities
    # The fitness value is the total distance traveled
    # Summing up the distances between continuous cities along the path
    distance_matrix = [
        [0, 2, INT_MAX, 12, 5],
        [2, 0, 4, 8, INT_MAX],
        [INT_MAX, 4, 0, 3, 3],
        [12, 8, 3, 0, 10],
        [5, INT_MAX, 3, 10, 0],
    ]
    fitness = 0
    for i in range(V - 1):
        fitness += distance_matrix[int(gnome[i])][int(gnome[i + 1])]
        # Add the distance from the last city back to the starting city
        fitness += distance_matrix[int(gnome[-1])][int(gnome[0])]
    return fitness

In [75]:
# Main function for TSP problem.
def tsp_util():
    global max_gen  # Declare max_gen as global

    # A list to store the population of individuals
    population = []

    # Current generation number
    gen = 1

    print("Initial population: \nGNOME FITNESS VALUE")
    # Populating the GNOME pool.
    for i in range(POP_SIZE):
        gnome = create_gnome()
        fitness = calculate_fitness(gnome)
        print(gnome, fitness)
        population.append(Individual(gnome, fitness))

    # Iteration to perform
    while gen <= max_gen:
        population.sort(key=lambda x: x.fitness)
        new_population = []

        # Elitism: Carry over the best individuals to the next generation
        elites_count = int(POP_SIZE * 0.5)  # Keep top 50% of the population as elites
        elites = population[:elites_count]
        new_population.extend(elites)

        # Selection, Crossover, and Mutation
        for _ in range((POP_SIZE - elites_count) // 2):
            parent1 = choice(population)
            parent2 = choice(population)
            child1_gnome, child2_gnome = crossover(parent1, parent2)
            mutated_gnome1 = mutated_gene(child1_gnome)
            mutated_gnome2 = mutated_gene(child2_gnome)
            new_population.append(Individual(mutated_gnome1, calculate_fitness(mutated_gnome1)))
            new_population.append(Individual(mutated_gnome2, calculate_fitness(mutated_gnome2)))

        # If the new population size is less than the original, add random individuals to match the population size
        while len(new_population) < POP_SIZE:
            gnome = create_gnome()
            new_population.append(Individual(gnome, calculate_fitness(gnome)))

        population = new_population

        print(f"\nGeneration {gen}")
        print("GNOME FITNESS VALUE")

        for ind in population:
            print(f"{ind.gnome} {ind.fitness}")

        gen += 1

    # Handle result
    # Find the minimum fitness value
    min_fitness = min(individual.fitness for individual in population)

    # Find all individuals with the minimum fitness value
    best_individuals = [individual for individual in population if individual.fitness == min_fitness]

    # Print the smallest fitness value (shortest path)
    print(f"\nShortest path (smallest fitness value in the last iteration): {min_fitness}")

    # Decode the gnome (visiting order) and print the unique solutions
    unique_results = set()

    for best_individual in best_individuals:
        best_gnome = best_individual.gnome
        result = "".join([GENES[int(city)] for city in best_gnome])  # List comprehension
        visiting_order = f"{result}{result[0]}"  # Add starting city at the end
        if visiting_order not in unique_results:
            unique_results.add(visiting_order)
            print(f"Visiting order (city names): {visiting_order}")

if __name__ == "__main__":
    tsp_util()

Initial population: 
GNOME FITNESS VALUE
13204 10737418251
34201 2147483694
20143 4294967318
41023 4294967339
21403 2147483680
32041 4294967334
31024 2147483700
23410 10737418250
31042 30
03142 10737418258

Generation 1
GNOME FITNESS VALUE
31042 30
21403 2147483680
34201 2147483694
31024 2147483700
20143 4294967318
23104 30
34021 2147483698
10243 2147483694
40213 2147483704
40312 41

Generation 2
GNOME FITNESS VALUE
31042 30
23104 30
40312 41
21403 2147483680
34201 2147483694
31240 68
40231 10737418251
40213 2147483704
43012 40
32041 4294967334

Generation 3
GNOME FITNESS VALUE
31042 30
23104 30
43012 40
40312 41
31240 68
01243 67
42103 61
12304 8589934612
02314 4294967325
10432 36

Generation 4
GNOME FITNESS VALUE
31042 30
23104 30
10432 36
43012 40
40312 41
13204 10737418251
03241 2147483673
30412 2147483680
21304 41
31042 30

Generation 5
GNOME FITNESS VALUE
31042 30
23104 30
31042 30
10432 36
43012 40
04123 2147483707
10342 43
10342 43
32401 45
20431 2147483686

Generation 6
GNOME 